In [1]:
# 共通で利用するライブラリ
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.3f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline
%load_ext autoreload

# 追加するライブラリ
from natsort import natsorted
import glob
import os
import datetime

%autoreload
from source.data_initialization import Initialize as ini
from source.calculation import Calc
from source.reporting import Reporting


## Ch.05 分析システムを構築する

データ一覧

- m_area.csv
  - 地域マスタ
  - 都道府県情報等
- m_store.csv
  - 店舗マスタ
  - 店舗名等
- tbl_order_202004.csv
  - 注文データ
  - 4月分
- tbl_order_202005.csv
  - 注文データ
  - 5月分
- tbl_order_202006.csv
  - 注文データ
  - 6月分
- tbl_order_202007.csv
  - 注文データ
  - 7月分

### Knock41: 基本的なフォルダを生成する

フォルダ構成

```bash
ch05
├── ch05.ipynb
├── source
└── data
    ├── 0_input
    ├── 10_output
    ├── 99_master
    └── race
```

In [2]:
# フォルダパスの定義

source_dir = "source"
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "10_output")
master_dir = os.path.join(data_dir, "99_master")
print(input_dir)


data/0_input


In [3]:
# フォルダの生成

os.makedirs(source_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

### Knock42: 入力データのチェック機能を作る

In [4]:
# マスタデータの読み込み

m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
print(m_area.shape)
display(m_area)

(7, 3)


,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉
3,SA,埼玉,埼玉
4,IB,北関東,茨城
5,TO,北関東,栃木
6,GU,北関東,群馬


In [5]:
# 指定した注文データ読み込み

trg_ym = "202007"
target_file = "tbl_order_" + trg_ym + ".csv"
target_data = pd.read_csv(os.path.join(input_dir, target_file))

In [6]:
# データチェック機構（正常動作時）

max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")

if (trg_ym == min_str_date) & (trg_ym == max_str_date):
    print("OK! 日付が一致しました。")
else:
    raise print("ERROR!! 日付が一致しません。")

OK! 日付が一致しました。


In [7]:
# 当月分の初期化

target_data = ini.init_tran_df(target_data, m_store, m_area)


### Knock43: レポーティング（本部向け）を関数化する

### Knock44: レポーティング（店舗向け）を関数化する

### Knock45: 関数を実行して動作を確認する

In [8]:
# 本部向けレポート

Reporting.make_report_headquarters(target_data, max_str_date, m_store, output_dir)

In [9]:
# 店舗向けレポート

for store_id in m_store.loc[m_store["store_id"] != 999]["store_id"]:
    Reporting.make_report_store(
        target_data, store_id, m_store, max_str_date, output_dir
    )


AttributeError: 'numpy.int64' object has no attribute 'loc'

In [ ]:
store_id = m_store.loc[m_store["store_id"] == 9]["store_id"]
store_id

8    9
Name: store_id, dtype: int64

In [ ]:
rank = Calc.get_store_rank(store_id, target_data, m_store)
rank

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/cm/w7d5zwgj0h3gp9q0lvvq4yrr0000gn/T/ipykernel_26399/2687411324.py", line 1, in <cell line: 1>
    rank = Calc.get_store_rank(store_id, target_data, m_store)
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/ch05/source/calculation.py", line 48, in get_store_rank
    store_rank = rank.loc[rank["store_id"] == target_id].index + 1
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/venv/lib/python3.9/site-packages/pandas/core/ops/common.py", line 70, in new_method
    return method(self, other)
  File "/Users/takeru/Library/CloudStorage/O

In [ ]:
store_sale = rank.loc[rank["store_id"] == store_id]["total_amount"]



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/cm/w7d5zwgj0h3gp9q0lvvq4yrr0000gn/T/ipykernel_26399/3675499334.py", line 2, in <cell line: 2>
    rank = Calc.get_store_rank(store_id, target_data, m_store)
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/ch05/source/calculation.py", line 48, in get_store_rank
    store_rank = rank.loc[rank["store_id"] == target_id].index + 1
  File "/Users/takeru/Library/CloudStorage/OneDrive-個人用/Learn/Python/python-practical-machine-learning-system-100knocks/venv/lib/python3.9/site-packages/pandas/core/ops/common.py", line 70, in new_method
    return method(self, other)
  File "/Users/takeru/Library/CloudStorage/O

In [ ]:
store_id
rank


NameError: name 'rank' is not defined